In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import torch
import re
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from transformers import RobertaTokenizer, RobertaForSequenceClassification, get_linear_schedule_with_warmup
from collections import defaultdict, Counter
from sklearn.preprocessing import LabelEncoder

c:\Users\chris\MyStuff\NUS\Y4S2\CS4248\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [4]:
# Read in json file
jsonObjTrain = pd.read_json(path_or_buf=r'train.jsonl', lines=True)
jsonObjTest = pd.read_json(path_or_buf=r'test.jsonl', lines=True)
jsonObjVal = pd.read_json(path_or_buf=r'dev.jsonl', lines=True)

# Select relevant columns
train_df = jsonObjTrain[['string', 'label']]
print("train_df\nRows: ", train_df.shape[0], "\n", train_df.head())
test_df = jsonObjTest[['string', 'label']]
print("\ntest_df\nRows: ", test_df.shape[0], "\n", test_df.head())
val_df = jsonObjVal[['string', 'label']]
print("\nval_df\nRows: ", val_df.shape[0], "\n", val_df.head())

,string,label
0,"Chapel, as well as X10 [2], UPC [3] , CoArray ...",background
1,"In addition, the result of the present study s...",result
2,Several instruments that more specifically add...,background
3,Organotypic hippocampal slice cultures\nInterf...,method
4,Activated PBMC are the basis of the standard P...,background
...,...,...
1856,"Additionally, no sex differences were detected...",result
1857,WBRT (40Gy in 20 fractions) along with concurr...,background
1858,The data obtained from this crosssectional sur...,method
1859,"Additionally, encapsulated spheroids may be mu...",background


In [10]:
def f(x):
    tv = torch.tensor(
        [
            tokenizer.encode(v, padding="max_length", max_length=500, truncation=True)
            for v in x
        ]
    ).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:, 1])  # use one vs rest logit units
    return val

explainer = shap.Explainer(f, tokenizer)

X_test = test_df["string"].tolist()
y_test = torch.tensor(test_df["label"].apply(lambda row: label_2_idx.index(row)).tolist())

# Compute SHAP values
shap_values = explainer(X_test, fixed_context=1, batch_size=2)

# Visualize SHAP values
shap.plots.text(shap_values)

None


TypeError: 'NoneType' object is not subscriptable

In [ ]:
label_2_idx = ['background', 'method', 'result']

X_test = test_df["string"].tolist()
y_test = torch.tensor(test_df["label"].apply(lambda row: label_2_idx.index(row)).tolist())

# Compute SHAP values
shap_values = explainer(X_test, fixed_context=1, batch_size=2)

# Visualize SHAP values
shap.plots.text(shap_values)